In [1]:
import os 
import pandas as pd
import json
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize, sent_tokenize
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession

/Users/lam.nguyen/Desktop/GithubClone/BigData_Final/.venv/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/lam.nguyen/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lam.nguyen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lam.nguyen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [82]:
num_topics = 5
max_iter = 10

# NLTK

In [1]:
# import zipfile
# import os

# # Path to the ZIP file
# zip_file_path = os.path.join("..", "data", "corpus.zip")

# # Destination folder to extract to
# destination_folder = os.path.join("..", "data", "raw")


# # Extract the ZIP file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(destination_folder)

# print("Extraction complete to:", destination_folder)


Extraction complete to: ../data/raw


In [5]:
# import os

# folder_path = os.path.join("..", "data", "raw")
# file_name = "test02.json"  # Replace with your target file name

# file_path = os.path.join(folder_path, file_name)

# # Check if file exists and delete it
# if os.path.exists(file_path) and os.path.isfile(file_path):
#     os.remove(file_path)
#     print(f"Deleted: {file_path}")
# else:
#     print("File does not exist.")

File does not exist.


In [3]:
# def merge_files() -> None:
#     """Merge all JSON files of a folder

#     Sample folder structure
#     ```
#     └── 📁data
#         └── 📁merged
#             └── test.json
#         └── 📁processed
#         └── 📁raw
#             └── test01.json
#             └── test02.json
#     ```
#     Args:
#         None
#     Return:
#         returns None"""
#     files = os.listdir(os.path.join("..", "data", "raw"))
#     text_todf = []
#     for i in range(len(files)):
#         with open(os.path.join("..", "data", "raw", files[i]), 'r') as file:
#             data = json.load(file)
#             df_metadata = pd.json_normalize(data, max_level=1)
#             text = ""
#             for element in df_metadata['body_text'][0]:
#                 text += element['text']
#             text_todf.append(text)
#     df = pd.DataFrame({"text": text_todf})
#     df.to_json(os.path.join("..", "data", "merged", "corpus.json"))

In [6]:
# # Merge files in data/raw
# from helper import merge_files
# merge_files()

In [73]:
import pandas as pd
data = pd.read_json(os.path.join("..", "data", "merged", "test.json"))

In [30]:
def custom_word_tokenize(sent_tokens: list[str]) -> list[str]:
    """Tokenises words 
    
    Args:
        sent_tokens: tokens of sentences
    Returns:
        A list of word tokens"""
    word_tokens = []
    for sent in sent_tokens:
        tokens = word_tokenize(sent)
        word_tokens.extend(tokens)
    return word_tokens

In [69]:
def preprocess(data: pd.DataFrame) -> pd.DataFrame:
    """Performs the preprocessing
    
    Args:
        data: input file as pandas.DataFrame
    Returns:
        A pandas.DataFrame which as vector form of the input
    """
    stop_words = stopwords.words('english')
    lemmatiser = WordNetLemmatizer()

    #Sentence tokenisation
    data['sent_tokens'] = data['text'].apply(sent_tokenize)
    
    # Text cleaning
    data['sent_tokens'] = data['sent_tokens'].apply(lambda sentences: [re.sub(r"[^a-zA-Z\s]", " ", sent).strip() for sent in sentences])
    
    # Normalisation
    data['sent_tokens'] = data['sent_tokens'].apply(lambda sentences: [sent.lower() for sent in sentences])

    #Word tokenisation
    data['word_tokens'] = data['sent_tokens'].apply(custom_word_tokenize)

    # # Stemming
    data['word_tokens'] = data['word_tokens'].apply(lambda word_tokens: [lemmatiser.lemmatize(word) for word in word_tokens])

    # Word Embeddings (turns into vector)
    data['to_tfidf'] = data['word_tokens'].apply(lambda tokens: ' '.join(tokens))
    vectorizer = TfidfVectorizer(min_df=0.3, max_df=0.85, stop_words=stop_words)
    tfidf_matrix = vectorizer.fit_transform(data['to_tfidf'])
    
    # Convert to DataFrame
    tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
    return data, tfidf

In [70]:
data, tfidf  = preprocess(data)

In [71]:
tfidf

,ability,able,academy,acceleration,access,accordance,according,account,accuracy,accurate,...,widely,window,work,working,world,worldwide,would,writing,wtft,wtfts
0,0.057213,0.028606,0.028606,0.028606,0.057213,0.028606,0.00000,0.00000,0.114426,0.028606,...,0.028606,0.028606,0.00000,0.00000,0.00000,0.000000,0.114426,0.00000,0.228852,0.028606
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04131,0.04131,0.000000,0.000000,...,0.000000,0.000000,0.04131,0.04131,0.04131,0.082619,0.000000,0.04131,0.000000,0.000000


# PySpark LDA

In [43]:
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession

In [10]:
# Create spark spession
spark = SparkSession.builder\
        .appName("Nhom09_PySparkLDA")\
        .master("local[*]")\
        .config("spark.driver.bindAddress", "localhost")\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/28 08:49:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Fit tfidf (panads.DataFrame) into data structure of spark
tfidf_ps = spark.createDataFrame(tfidf)
tfidf_ps = tfidf_ps.rdd.map(lambda row: (Vectors.dense(row), )).toDF(["features"])

In [45]:
from pyspark.ml.clustering import LDA

# Create an LDA model
lda = LDA(k=num_topics, maxIter=max_iter)  # k is the number of 
# Fit the LDA model
lda_model = lda.fit(tfidf_ps)
# Describe topics
topics = lda_model.describeTopics(3)  # Get the top 3 terms for each topic
topics.show(truncate=False)

+-----+---------------+---------------------------------------------------------------------+
|topic|termIndices    |termWeights                                                          |
+-----+---------------+---------------------------------------------------------------------+
|0    |[404, 41, 105] |[0.002398466433135447, 0.002334772230784771, 0.0023063777294193063]  |
|1    |[343, 164, 35] |[0.0023859780701145855, 0.0023294478680108786, 0.0023203198160067742]|
|2    |[108, 8, 232]  |[0.0024689895728190795, 0.002377024731400353, 0.0023564226010567954] |
|3    |[176, 220, 273]|[0.0023427878938911107, 0.0023394471897149522, 0.0023343600894544132]|
|4    |[26, 196, 278] |[0.0024366442857787266, 0.002359836168195985, 0.002356088139912443]  |
+-----+---------------+---------------------------------------------------------------------+



# SparkNLP

In [86]:
import sparknlp
import os
import pandas as pd
import pyspark
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA

In [2]:
sparknlp_session = sparknlp.start(params={"spark.driver.host": "localhost",
                                          "spark.driver.port": "9999",
                                          "spark.driver.bindAddress": "127.0.0.1"})

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", sparknlp_session.version)

:: loading settings :: url = jar:file:/Users/lam.nguyen/Desktop/GithubClone/BigData_Final/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/lam.nguyen/.ivy2/cache
The jars for the packages stored in: /Users/lam.nguyen/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5a92852d-f11f-45bd-89e7-379d8bc12258;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;6.0.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500

Spark NLP version:  6.0.0
Apache Spark version:  3.5.5


In [3]:
# Read data
corpus = pd.read_json(os.path.join("..", "data", "merged", "test.json"))
corpus

,text
0,As a consequence of the global COVID-19 pandem...
1,According to current live statistics at the ti...


In [84]:
lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features')
lda_model = lda.fit(tfidf_result)
topics = lda_model.describeTopics(3)  # Get the top 3 terms for each topic
topics.show(truncate=False)

+-----+---------------+---------------------------------------------------------------------+
|topic|termIndices    |termWeights                                                          |
+-----+---------------+---------------------------------------------------------------------+
|0    |[403, 3, 395]  |[0.00229576121278365, 0.0022629753171704396, 0.0021983315604854497]  |
|1    |[145, 206, 243]|[0.002363763783625437, 0.002311296300993184, 0.0022137760633377515]  |
|2    |[62, 151, 546] |[0.0022826458617294563, 0.0022737865312271468, 0.0022717844376154207]|
|3    |[490, 349, 76] |[0.002365765042149912, 0.0022072434298510374, 0.0022055900246234204] |
|4    |[448, 548, 317]|[0.002326554154138089, 0.0023005053558257226, 0.0022530961892851076] |
+-----+---------------+---------------------------------------------------------------------+

